In [2]:
import os
from os.path import isfile, join, abspath
import datetime
import csv
import pandas as pd
import shutil
from pathlib import Path
# from azure.storage.blob import BlobServiceClient
from azure.storage.blob import ContainerCliente
import snowflake.connector as sf
from snowflake.connector.pandas_tools import write_pandas
import win32com.client as win32

# 1. Analyst Input info  ######################################################################
weekly_record_move_file = 

def upload_SF(df_tracker):
    """Set up connection to SnowFlake"""
    # account = "ih73640.australia-east.azure"
    # user = "duongnguyen@profectusgroup.com"
    # warehouse = "W1"
    # role = "PVN_AUDIT_COLES"
#################################################################################################
    account = "ih73640.australia-east.azure"
    user = "tdinh@profectusgroup.com"
    warehouse = "W1"
    role = "PVN_AUDIT_LEADS"
    database = "DEMO_DB"
    schema = "CATALOGUE"
    password = ""
    auth = "externalbrowser"

    conn = sf.connect(user=user, password=password, account=account, authenticator=auth,
                      warehouse=warehouse, role=role, database=database, schema=schema)
    print('start loading data')
    success, nchunks ,nrows , _ = write_pandas(conn,df_tracker,'CATALOGUE_TRACKING',compression = 'gzip')
    print(str(success)+', '+str(nchunks)+', '+str(nrows))
    print('done loading data')
    conn.close()
    return None

container_client = ContainerClient.from_container_url(sas_url)

def find_path_week():
    today = datetime.date.today()
    if (today.weekday() == 0):
        monday = today + datetime.timedelta( (7-today.weekday()) % 7)
    else:
        monday = today + datetime.timedelta( (7-today.weekday()) % 7 -7)
    cata_date = monday.strftime("%Y%m%d")
    cata_date_year = cata_date[0:4]
    cata_date_month = cata_date[4:6]
    cata_date_day = cata_date[2:]
    return fr'D:\OneDrive - Profectus Group\Ongoing Catalogue\Auto\Tool\catalogues\{cata_date_year}\{cata_date_month}\{cata_date_day}'



path_week = find_path_week()
cat_date= '20' + path_week.split('\\')[-1]
cat_year = cat_date[0:4]

# list all file in folder
dict_ven_filename = {}
for (dir_path, dir_names, file_names) in os.walk(path_week):
    for file_name in file_names:
        vendor_name = file_name.split('_')[0]
        dict_ven_filename[join(path_week,file_name)] = vendor_name

df_vendor_transfer = pd.read_excel(r'vendor_transfer.xlsx')
#transfer link
def transfer_link_sharepoint(df_vendor_transfer,dict_ven_filename):
    output_path = join('D:\OneDrive - Profectus Group\Ongoing Catalogue','vendor_transfer',cat_year,cat_date)
    dict_vendor_transfer = df_vendor_transfer.set_index('Retailer_name')['Vendor_path'].to_dict()
    dict_link_transfer = {}
    for file_name , vendor_name in dict_ven_filename.items():
        if vendor_name in dict_vendor_transfer.keys():
            dict_link_transfer[file_name] = output_path.replace('vendor_transfer',dict_vendor_transfer[vendor_name])
    return dict_link_transfer


def transfer_link_azure_storage(df_vendor_transfer,dict_ven_filename):
    output_path = join('vendor_transfer',cat_year,cat_date).replace('\\','/')
    dict_vendor_transfer = df_vendor_transfer.set_index('Retailer_name')['Vendor_path'].to_dict()
    dict_link_transfer = {}
    for file_name , vendor_name in dict_ven_filename.items():
        if vendor_name in dict_vendor_transfer.keys():
            dict_link_transfer[file_name] = output_path.replace('vendor_transfer',dict_vendor_transfer[vendor_name])
    return dict_link_transfer

def copy_file_sharepoint(dict_link_transfer_sharepoint):
    with open(weekly_record_move_file, "a", encoding="utf-8", newline="") as weekly_record_io:
        weekly_record_writer = csv.writer(weekly_record_io)     
        for source_path, des_path in dict_link_transfer_sharepoint.items():
            # print(source_path, des_path)
            Path(des_path).mkdir(parents=True, exist_ok=True)
            if (isfile(join(des_path,source_path.split('\\')[-1])) == False):
                des_path = join(des_path,source_path.split('\\')[-1])
                print(source_path, des_path)
                # print('new')
                try:
                    shutil.copy2(source_path,des_path)
                    status = 'Done'
                except:
                    status = 'Fail'
                now_dtt = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
                weekly_record_writer.writerow([path_week.split('\\')[-1],source_path,des_path,status, now_dtt])
    return True

def copy_file_azure(dict_link_transfer_azure):
    for source_path, des_path in dict_link_transfer_azure.items():
        des_path = join(des_path,source_path.split('\\')[-1])
        # Open the file in read mode
        print(source_path,des_path)
        with open(source_path, 'rb') as data:
            # Get a reference to the blob
            blob_client = container_client.get_blob_client(des_path)
            # Upload the file
            blob_client.upload_blob(data, overwrite=True)
        print(f'Done {des_path}')
    return True

dict_link_transfer_azure = transfer_link_azure_storage(df_vendor_transfer,dict_ven_filename)
dict_link_transfer_sharepoint = transfer_link_sharepoint(df_vendor_transfer,dict_ven_filename)

def df_line_tracker(des_path,cat_date):    
    des_path_list = des_path.split('/')[-1].split('.')[-2].split('_')
    cata_name = des_path_list[-1]
    client_name = des_path_list[0]
    promo_start = des_path_list[1]
    if len(des_path_list) == 3:
        promo_end = ''
    else:
        promo_end = des_path_list[2]
    des_path = des_path.replace(' ','%20')
    return pd.DataFrame([[client_name,cata_name,promo_start,promo_end,cat_date,des_path]],columns = ['CLIENT_NAME','FILE_NAME','PROMO_START','PROMO_END','RETRIEVAL_DATE','FILE_PATH'])

def upload_df_catalogue_tracking(dict_link_transfer_azure,cat_date):
    df_tracker = pd.DataFrame(columns = ['CLIENT_NAME','FILE_NAME','PROMO_START','PROMO_END','RETRIEVAL_DATE','FILE_PATH'])
    for source_path, des_path in dict_link_transfer_azure.items():
        # print(source_path, des_path)
        des_path = "https://pvnadfstorage.blob.core.windows.net/catalogue/"+join(des_path,source_path.split('\\')[-1]).replace('\\','/')
        # df_tracker 
        df_line = df_line_tracker(des_path,cat_date)
        df_tracker = pd.concat([df_tracker.reset_index(drop=True),df_line.reset_index(drop=True)]).reset_index(drop=True)
    upload_SF(df_tracker)
    return df_tracker

def upload_df_no_catalogue(cat_date,dict_ven_filename):
    client_name_set = set()
    for file in dict_ven_filename.keys():
        client_name_set.add(file.split('\\')[-1].split('_')[0]) 
    client_name_list = list(client_name_set)
    df_tracker_no_cata = pd.DataFrame(columns = ['CLIENT_NAME','FILE_NAME','PROMO_START','PROMO_END','RETRIEVAL_DATE','FILE_PATH'])
    client_name_list_full = df_vendor_transfer['Retailer_name'].to_list()
    for client_name in client_name_list_full:
        if client_name not in client_name_list:
            df_line = pd.DataFrame([[client_name,'No new catalogue available',None,None,cat_date,None]],columns = ['CLIENT_NAME','FILE_NAME','PROMO_START','PROMO_END','RETRIEVAL_DATE','FILE_PATH'])
            df_tracker_no_cata = pd.concat([df_tracker_no_cata.reset_index(drop=True),df_line.reset_index(drop=True)]).reset_index(drop=True)
    upload_SF(df_tracker_no_cata)
    return df_tracker_no_cata




In [ ]:
# 2. Move file to SharePoint and Azure
copy_file_sharepoint(dict_link_transfer_sharepoint)
copy_file_azure(dict_link_transfer_azure)

In [ ]:
# 3. Update to Snowflake
upload_df_catalogue_tracking(dict_link_transfer_azure,cat_date)
upload_df_no_catalogue(cat_date,dict_ven_filename)

In [30]:
# 4. Send email
def send_email(cat_date):
    ol=win32.Dispatch("outlook.application")
    olmailitem=0x0 #size of the new email
    newmail=ol.CreateItem(olmailitem)
    newmail.Subject= f'Ongoing Catalogue Task - Date: {cat_date}'
    # newmail.To='tdinh@profectusgroup.com'
    newmail.To=
    newmail.CC=
    newmail.BodyFormat = 2 
    newmail.HTMLBody = f'''
    <p style="font-family:Arial; font-size:10pt;">Hi Matt & chi Hanh,<br><br>The catalogues for {cat_date} have been collected and stored on SharePoint.</p>

    '''
    newmail.Send()
    return None
send_email(cat_date)

In [10]:
find_path_week()

'D:\\OneDrive - Profectus Group\\Ongoing Catalogue\\Auto\\Tool\\catalogues\\2024\\06\\240617'